In [5]:
import os
import pandas as pd
import folium
from folium.plugins import MarkerCluster


def load_agences(folder: str, bank_name: str) -> pd.DataFrame:
    """Lit tous les CSV d'un dossier, conserve les colonnes lat/lng, ajoute la colonne 'bank'."""
    dfs = []
    if not os.path.isdir(folder):
        print(f"⚠️ Dossier introuvable : {folder}")
        return pd.DataFrame()

    for fn in os.listdir(folder):
        if not fn.lower().endswith('.csv'):
            continue
        path = os.path.join(folder, fn)
        try:
            df = pd.read_csv(path, encoding='utf-8-sig')
        except Exception as e:
            print(f"⚠️ Erreur lecture '{path}': {e}")
            continue

        # Normalise les noms de colonnes
        df.columns = [c.strip().lower() for c in df.columns]

        # Vérifie la présence de latitude/longitude
        if not {'latitude', 'longitude'}.issubset(df.columns):
            continue

        # Nettoie et convertit
        df = df.dropna(subset=['latitude', 'longitude'])
        df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
        df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
        df = df.dropna(subset=['latitude', 'longitude'])

        # Ajoute la colonne banque
        df['bank'] = bank_name
        dfs.append(df)

    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()


if __name__ == '__main__':
    # ► Charger les données géocodées
    bnp_folder = os.path.join('scraper', 'agences_bnp')
    sg_folder  = os.path.join('scraper', 'agences_sg_geocoded')

    df_bnp = load_agences(bnp_folder, 'BNP Paribas')
    df_sg  = load_agences(sg_folder, 'Société Générale')

    # ► Statistiques
    print(f"– BNP : {len(df_bnp)} agences")
    print(f"– SG  : {len(df_sg)} agences")
    df_all = pd.concat([df_bnp, df_sg], ignore_index=True)
    print(f"– Total : {len(df_all)} agences")

    # ► Initialisation de la carte
    if not df_all.empty:
        center = df_all[['latitude', 'longitude']].mean().tolist()
    else:
        center = [46.5, 2.5]
    m = folium.Map(location=center, zoom_start=6, control_scale=True)

    # ► Groupes / clusters avec couleurs personnalisées
    colors = {'BNP Paribas': '#8A2BE2', 'Société Générale': 'red'}
    for bank_name, df in [('BNP Paribas', df_bnp), ('Société Générale', df_sg)]:
        fg = folium.FeatureGroup(name=bank_name, show=(bank_name == 'BNP Paribas'))
        marker_cluster = MarkerCluster().add_to(fg)
        color = colors.get(bank_name, 'blue')
        for _, row in df.iterrows():
            folium.CircleMarker(
                location=(row['latitude'], row['longitude']),
                radius=5,
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=0.7
            ).add_to(marker_cluster)
        fg.add_to(m)

    # ► Contrôle des couches
    folium.LayerControl(collapsed=False).add_to(m)

    # ► Sauvegarde du rendu
    out_file = 'carte_agences_bnp_sg.html'
    m.save(out_file)
    print(f"🗺️ Carte multi-banques exportée : {out_file}")


– BNP : 2210 agences
– SG  : 1837 agences
– Total : 4047 agences
🗺️ Carte multi-banques exportée : carte_agences_bnp_sg.html


In [5]:
import pandas as pd
import time
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# 1. Charge ton CSV SG
df = pd.read_csv("scraper/agences_sg/agences-ain-01.csv", encoding="utf-8-sig")

# 2. Prépare le géocodeur (rate‑limit 1 req/sec)
geolocator = Nominatim(user_agent="my_geocoder")
geocode    = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# 3. Construit l’adresse à passer au géocodeur : adresse + code postal + France
df["full_address"] = df["adresse"] + ", " + df["code_postal"].astype(str) + ", France"

# 4. Géocode
df["location"]  = df["full_address"].apply(geocode)
df["latitude"]  = df["location"].apply(lambda loc: loc.latitude  if loc else None)
df["longitude"] = df["location"].apply(lambda loc: loc.longitude if loc else None)

# 5. Sauvegarde
df.drop(columns=["full_address","location"], inplace=True)
df.to_csv("scraper/agences_sg_with_coords.csv", index=False, encoding="utf-8-sig")

print(df[["adresse","code_postal","latitude","longitude"]].head())


RateLimiter caught an error, retrying (0/2 tries). Called with (*('15 PL BERNARD, 01000 BOURG EN BRESSE, 1000, France',), **{}).
Traceback (most recent call last):
  File "C:\Users\iroum\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py", line 537, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "C:\Users\iroum\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connection.py", line 461, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\http\client.py", line 1395, in getresponse
    response.begin()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Li

KeyboardInterrupt: 